In [3]:
%load_ext autoreload

import Request
import Rule
import itertools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
req = Request.Request()
rule = Rule.Rule('rule.conf')

In [5]:
host_values = ['xxx123.tk','xxx456.tk']
connection_values = ['Keep-Alive']

values = {}
values['Host'] = host_values
values['Connection'] = connection_values
values

{'Connection': ['Keep-Alive'], 'Host': ['xxx123.tk', 'xxx456.tk']}

In [6]:
def yield_values_permutation(values):
    value_dict = {}
    for header_key in values.keys():
        header_value = values.get(header_key)
        header_value_permutation = list(set(list(itertools.permutations(header_value,len(header_value)))))
        value_dict[header_key] = header_value_permutation
    
    return value_dict

In [7]:
header_value_dict = yield_values_permutation(values)
header_value_dict

{'Connection': [('Keep-Alive',)],
 'Host': [('xxx456.tk', 'xxx123.tk'), ('xxx123.tk', 'xxx456.tk')]}

In [8]:
def yield_header_value_and_style(rule,header_value_dict):
    headers  = {}
    rules = rule.get_rule()
    for header_key in rules.keys():
        headers[header_key] = []
        for header_style in rules.get(header_key):
            for header_value in header_value_dict.get(header_key):
                headers[header_key].append((header_value,header_style))
    
    return headers

In [9]:
header_info = yield_header_value_and_style(rule,header_value_dict)
header_info

{u'Connection': [(('Keep-Alive',), (None,))],
 u'Host': [(('xxx456.tk', 'xxx123.tk'),
   (u'space_succeeded', u'space_preceded')),
  (('xxx123.tk', 'xxx456.tk'), (u'space_succeeded', u'space_preceded')),
  (('xxx456.tk', 'xxx123.tk'), (u'space_preceded', u'space_succeeded')),
  (('xxx123.tk', 'xxx456.tk'), (u'space_preceded', u'space_succeeded'))]}

In [12]:
def yield_requests(request,header_info):
    requests = []
    header_lst = header_info.keys()        #由header名构成的列表
    header_info_lst = header_info.values() #由字典header_info中各header对应的值的列表
    
    
    for headers in itertools.product(*header_info_lst):  #heasers 不同Header字段的组合
        for i,multi_header in enumerate(headers):        #multi_header  多个同名的Header的值和style
            #multi_header[0]:多个同名的Header的值
            #multi_header[1]:多个同名的Header的style
            for j in range(len(multi_header[0])):
                request.set_Header(header_lst[i],multi_header[0][j],multi_header[1][j])
        
        requests.append(request.request())
        request.clear_headers()
             
    return requests

In [13]:
requests = yield_requests(req,header_info)

In [14]:
for request in requests:
    print request
    print '\n'

GET / HTTP/1.1
Host: xxx456.tk
 Host:xxx123.tk
Connection:Keep-Alive




GET / HTTP/1.1
Host: xxx123.tk
 Host:xxx456.tk
Connection:Keep-Alive




GET / HTTP/1.1
 Host:xxx456.tk
Host: xxx123.tk
Connection:Keep-Alive




GET / HTTP/1.1
 Host:xxx123.tk
Host: xxx456.tk
Connection:Keep-Alive




